In [1]:
import pandas as pd

rb = pd.read_excel('/home/aistudio/data/data60829/Online Retail.xlsx')

ImportError: Install xlrd >= 0.9.0 for Excel support

In [ ]:
rb.dropna(axis=0, subset=['InvoiceNo'], inplace=True) #删除缺失值
rb['InvoiceNo'] = rb['InvoiceNo'].astype('str')#类型转换为字符型
rb['StockCode'] = rb['StockCode'].astype('str')
rb = rb[~rb['InvoiceNo'].str.contains('C')]

In [ ]:
def createC1(dataSet):  # 创造候选项集C1，C1是大小为1的所有候选项集的集合
    C1 = []
    for transaction in dataSet:   
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    return list(map(frozenset,C1))
 
def scanD(D,Ck,minSupport):  # 此函数计算支持度,筛选满足要求的项集成为频繁项集Lk，D是数据集，Ck为候选项集C1或C2或C3 ...
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems 
        if support >= minSupport: 
            retList.append(key)
        supportData[key] = support
    return retList, supportData
 
def aprioriGen(Lk, k):   # 合并到更高层
    lenLk = len(Lk)
    temp_dict = {}  # 临时字典，存储 
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = Lk[i]|Lk[j]  # |用于集合的合并
            if len(L1) == k:  # 如果合并后的子项元素有k个，满足要求
                if not L1 in temp_dict:  # 把符合的新项存到字典的键中，使用字典可以去重复，比如{1,2,3}和{3，1，2}是一样的项，使用了字典就可以达到去重的作用
                    temp_dict[L1] = 1
    return list(temp_dict)  # 把字典的键转化为列表
 
def apriori(dataSet, minSupport = 0.05):  
    C1 = createC1(dataSet)
    D = list(map(set,dataSet))
    L1,supportData = scanD(D, C1, minSupport) # 筛选出L1
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):   # 创造Ck
        Ck = aprioriGen(L[k-2],k)
        Lk, supK = scanD(D,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L,supportData

def calcConf(freqSet, H, supportData, br1, minConf = 0.7): # 筛选符合可信度要求的规则，并返回符合可信度要求的右件
    prunedH = []  # 存储符合可信度的右件
    for conseq in H:  # conseq就是右件，freqSet是原始频繁项,freqSet-conseq是左件
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf>= minConf:
            print(freqSet-conseq,"-->",conseq,"\tconf:",conf)
            br1.append((freqSet-conseq,conseq,conf))
        else:
            prunedH.append(conseq) # 不符合可信度的右件添加到列表中
    return prunedH

def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7): # 新版Apriori原理来减少创造的规则
    is_find = True  # 循环标志
    m = 1 # 先创造右件为一个元素的规则
    Hmp1 = H   #[frozenset({2}),frozenset({3}),frozenset({5)],Hmp1是组合后的右件，右件为1个元素的时候我们直接H赋值过去，当右件元素数是2以上的时候，用aprioriGen组合出来
    while is_find:
        if len(freqSet) > m:
            if m > 1: # 我们改编的aprioriGen()函数至少产生C2,不能产生C1，因此这里加了if
                Hmp1 = aprioriGen(H,m)  # H里的元素自由组合成右件，右件的元素个数是m
            H_no = calcConf(freqSet, Hmp1, supportData, br1, minConf) # 筛选符合可信度的规则,把不符合的右件存起来
            if len(H_no) != 0: # 如果有不满足可信度的右件
                H_no = list(set(frozenset([item]) for row in Hmp1 for item in row)) # [{2,3},{3,4}] 分割后为[{2},{3},{4}]，这么操作就是把不满足的右件及其超集提出来，然后后面做减法。
                H = list(set(H)-set(H_no))  # 可组合的集合减去不满足可信度的右件的集合????????????
            m = m + 1
            if len(H) < m: 
                is_find = False
        else: 
            is_find = False
 
def generateRules(L, supportData, minConf=0.7):  # 产生规则
    bigRuleList = []
    for i in range(1, len(L)):  # 从L2开始创造规则
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if i>1:  # L3开始使用Apriori原理
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:  # L2不能使用Apriori原理，只能老老实实挨个创造规则
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList


In [ ]:
l1 = list(rb.loc[(rb['Country'] == 'France')& (rb['StockCode']!='POST')]['InvoiceNo'])
l2 = list(rb.loc[(rb['Country'] == 'France')& (rb['StockCode']!='POST')]['StockCode'])
data = []
tt = []
for i in range(0, len(l1)-1):
    if l1[i]==l1[i+1]:
        tt.append(l2[i])
    else:
        tt.append(l2[i])
        data.append(tt)
        tt = []


In [ ]:
if __name__ == "__main__":
    dataSet = data
    L,suppData = apriori(dataSet)
    print(L)    #所有频繁项集

    rules = generateRules(L, suppData, minConf=0.7)    #所有规则

In [ ]:
print(rb.loc[(rb['StockCode'] == '22551')]['Description'].head(1))
print(rb.loc[(rb['StockCode'] == '22554')]['Description'].head(1))
print()
print(rb.loc[(rb['StockCode'] == '21086')]['Description'].head(1))
print(rb.loc[(rb['StockCode'] == '21094')]['Description'].head(1))
print()
print(rb.loc[(rb['StockCode'] == '22728')]['Description'].head(1))
print(rb.loc[(rb['StockCode'] == '22726')]['Description'].head(1))
print(rb.loc[(rb['StockCode'] == '22727')]['Description'].head(1))